In [1]:
import Pkg
import DelimitedFiles
# Pkg.add("StatsKit")
# Pkg.add("random")
using StatsKit
using Random
using DelimitedFiles

In [2]:
struct mtDNA 
    r::Float64
    d::Float64
    unique_id::Int
    parent_id::Int
    
    function mtDNA(unique_id,parent_id)
        new(0.01,0.01,unique_id, parent_id)
    end
end 

In [3]:
rep(mol::mtDNA) = mol.r
deg(mol::mtDNA) = mol.d
unique_id(mol::mtDNA) = mol.unique_id
parent_id(mol::mtDNA) = mol.parent_id

parent_id (generic function with 1 method)

In [4]:
C_0 = 100
Nsim = 10000
Nout = 100
dt = Nsim/Nout;

In [7]:
system_state = [mtDNA(x,-1) for x=1:C_0 ]
current_id = C_0 + 1

output = Vector{Int}()
append!(output, length(system_state))

for i = 1:Nsim
    if i%dt==0
        append!(output, length(system_state))
    end
    
    molecules_to_remove = Vector{Int}()
    new_molecules = Vector{mtDNA}()
    
    for mol_ind=1:length(system_state)
        molecule = system_state[mol_ind]
        
        roll = rand(Float64)
        cdf = cumsum( [deg(molecule), rep(molecule)] ) 
        if 0.0<roll && roll<=cdf[1] # degredation
            append!(molecules_to_remove, mol_ind)
            
        elseif cdf[1]<roll && roll<=cdf[2] # rplication
            append!( molecules_to_remove, mol_ind)
            for j=1:2
                current_id += 1
                daughter = mtDNA(current_id, unique_id(molecule))
                push!(new_molecules, daughter)
            end
        end
    end
    system_state = [mol for (i,mol) in enumerate(system_state) if i ∉ molecules_to_remove ]
    append!(system_state, new_molecules)
end

In [8]:
output

101-element Vector{Int64}:
 100
  93
  81
  89
  68
  71
  77
  68
  55
  51
  50
  35
  46
   ⋮
 116
 143
 174
 181
 193
 231
 229
 264
 291
 323
 337
 318

In [8]:
open("ABMsim.txt", "w") do file
    write(file, output)
end

1616

In [9]:
writedlm("ABMsim.txt", output)